## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [32]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import seaborn as sns
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
config = pn.Config('master', 'V', 15)

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data
There is an error in the supplied ticker symbol EEFT in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company uses non stardard quarterly reporting date not currently supported. Company was dropped from the list.
There is an error in the supplied ticker symbol PAGS in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company was not found and was dropped from the list.


[autoreload of multiples failed: Traceback (most recent call last):
  File "/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 459, in superreload
    module = reload(module)
  File "/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/daved/Dev/pinon/pinon/multiples.py", line 9, in <module>
    PE_COLS = [pn_cols.QTR_EPS, pn_cols.QTR_REV, pn_cols.QTR_DIV, pn_cols.TTM_EPS, pn_cols.QTR_PE_RATIO, pn_cols.TTM_PE_RATIO]
AttributeError: m

In [68]:
c = pn.Comps(config)
c.run()
c.comp_ratios

KeyboardInterrupt: 

In [67]:
c.extend_forecasts()

KeyboardInterrupt: 

In [ ]:
c.calc_fair_value()
c.fair_value.loc['V']
c.fair_value

In [ ]:
c.target_ks.loc[('V',), :].plot()